In [1]:
import os
from keras import Sequential,Model
from keras.layers import Dense,Flatten,MaxPooling2D,Convolution2D,Activation,BatchNormalization,Softmax,Dropout,GlobalMaxPool2D,GlobalAveragePooling2D
from keras import optimizers
import numpy as np
import sklearn
import random
from pathlib import Path


import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.applications.densenet import DenseNet121



Using TensorFlow backend.


In [2]:
p = Path("./cell_images/")
dirs=p.glob('*')
label_dict={'Uninfected':0,'Parasitized':1}
label2class={0:'Uninfected',1:'Parasitized'}
image_data=[]
labels=[]
image_paths=[]
image_data = []
labels = []

#Optional
image_paths = []

for folder_dir in dirs:
    label = str(folder_dir).split("/")[-1]
    cnt = 0
    #Iterate over folder_dir
    for img_path in folder_dir.glob("*.png"):
        
        img = image.load_img(img_path,target_size=(100,100))
        img_array= image.img_to_array(img)
        image_data.append(img_array)
        labels.append(label_dict[label])
        cnt += 1
        
    print(cnt)

13779
13779


In [0]:
combined = list(zip(image_data,labels))
random.shuffle(combined)
image_data[:],labels[:] = zip(*combined)
x=np.array(image_data)
y=np.array(labels)
x=x/255.0


In [4]:
from keras.utils import np_utils
x_train=x[:int(x.shape[0]*0.8),:]
y_train=y[:int(y.shape[0]*0.8)]
x_test=x[x_train.shape[0]:x.shape[0],:]
y_test=y[y_train.shape[0]:y.shape[0]]
y_test=np_utils.to_categorical(y_test)
y_train=np_utils.to_categorical(y_train)
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

(22046, 100, 100, 3) (22046, 2) (5512, 100, 100, 3) (5512, 2)


In [5]:
res_model = DenseNet121(include_top=False,weights='imagenet', input_shape=(100,100,3))

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
av1 = GlobalAveragePooling2D()(res_model.output)
fc1 = Dense(256, activation='relu')(av1)
d1 = Dropout(0.25)(fc1)
out = Dense(2, activation='softmax')(d1)

tl_model = Model(inputs=res_model.input,outputs=out)
tl_model.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100, 100, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 106, 106, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 50, 50, 64)   9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 50, 50, 64)   256         conv1/conv[0][0]            

In [0]:
adam = optimizers.adam(lr = 0.0003,decay=0.00003)

In [0]:
tl_model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [9]:
hist = tl_model.fit(x_train,y_train,epochs=5,shuffle=True,batch_size=64,validation_split=0.20)

Instructions for updating:
Use tf.cast instead.
Train on 17636 samples, validate on 4410 samples
Epoch 1/5
17636/17636 [==============================] - 167s 9ms/step - loss: 0.2401 - acc: 0.9062 - val_loss: 0.1466 - val_acc: 0.9515
Epoch 2/5
17636/17636 [==============================] - 137s 8ms/step - loss: 0.0913 - acc: 0.9660 - val_loss: 0.1250 - val_acc: 0.9571
Epoch 3/5
17636/17636 [==============================] - 137s 8ms/step - loss: 0.0532 - acc: 0.9803 - val_loss: 0.1293 - val_acc: 0.9567
Epoch 4/5
17636/17636 [==============================] - 137s 8ms/step - loss: 0.0320 - acc: 0.9885 - val_loss: 0.1285 - val_acc: 0.9608
Epoch 5/5
17636/17636 [==============================] - 137s 8ms/step - loss: 0.0186 - acc: 0.9942 - val_loss: 0.1413 - val_acc: 0.9599


In [10]:
tl_model.evaluate(x_test,y_test)

5512/5512 [==============================] - 14s 3ms/step


[0.14099616902719517, 0.9615384615384616]

In [11]:
y_predicted=tl_model.predict(x_test)
y_predicted=np.argmax(y_predicted,axis=1)
print(y_predicted)

[1 1 1 ... 0 0 0]


In [0]:
y_test_score=y[y_train.shape[0]:y.shape[0]]

In [14]:

from sklearn.metrics import classification_report
target_names = ['class 1', 'class 2']

print(classification_report(y_test_score, y_predicted, target_names=target_names))

              precision    recall  f1-score   support

     class 1       0.95      0.97      0.96      2796
     class 2       0.97      0.95      0.96      2716

   micro avg       0.96      0.96      0.96      5512
   macro avg       0.96      0.96      0.96      5512
weighted avg       0.96      0.96      0.96      5512



In [15]:
from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(y_test_score, y_predicted)
print(confusion)

[[2723   73]
 [ 139 2577]]


In [0]:
TP = confusion[1, 1]

# True Negatives
TN = confusion[0, 0]

# False Positives
FP = confusion[0, 1]

# False Negatives
FN = confusion[1, 0]

In [17]:
###precision/sensitivity
print(TP/float(TP+FP))

0.9724528301886792


In [18]:
###recall/specificity
print(TP/float(TP+FN))
      

0.948821796759941
